In [ ]:
import pandas as pd
import numpy as np

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import string
import re

from numpy import array
from numpy import asarray
from numpy import zeros

from sklearn.model_selection import train_test_split

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

train = pd.read_csv('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

id='1ko2_L5ugKZLaZgvfYvNcYqurtGR8FZ-p'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('glove.6B.100d.txt')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
def remove_punct(text):
    #filtrado de signos de puntuacion
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    return text

In [ ]:
y = train.target
#train['text'] = train['text'].apply(lambda x: remove_punct(x))
x_corpus, x_validation, y_corpus, y_validation = train_test_split(train.text, y, test_size=0.3, stratify=y)

In [ ]:
corpus = list(x_corpus)

In [ ]:
len(corpus)

5329

In [ ]:
#DEL TRAIN SPLITEADO

#buscamos vocabulary (palabras unicas)
word_tokenizer_train = Tokenizer()
word_tokenizer_train.fit_on_texts(corpus)

#cantidad de palabras unicas
vocab_length = len(word_tokenizer_train.word_index) + 1

#convertimos los tweets a numeros
embedded_sentences_train = word_tokenizer_train.texts_to_sequences(corpus)

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
from nltk.tokenize import word_tokenize

word_count_train = lambda sentence: len(word_tokenize(sentence))
longest_sentence_train = max(corpus, key=word_count_train)
length_long_sentence_train = len(word_tokenize(longest_sentence_train))
padded_sentences_train = pad_sequences(embedded_sentences_train, length_long_sentence_train, padding='post')

In [ ]:
len(padded_sentences_train)

5329

In [ ]:
#convertimos los tweets DE VALIDACION a numeros
word_tokenizer_val = Tokenizer()
word_tokenizer_val.fit_on_texts(list(x_validation))
embedded_sentences_val = word_tokenizer_val.texts_to_sequences(list(x_validation))

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
word_count_val = lambda sentence: len(word_tokenize(sentence))
longest_sentence_val = max(corpus, key=word_count_val)
length_long_sentence_val = len(word_tokenize(longest_sentence_val))
padded_sentences_val = pad_sequences(embedded_sentences_val, length_long_sentence_val, padding='post')

In [ ]:
#convertimos los tweets DEL TEST a numeros
word_tokenizer_test = Tokenizer()
word_tokenizer_test.fit_on_texts(list(test.text))
embedded_sentences_test = word_tokenizer_test.texts_to_sequences(list(test.text))

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
word_count_test = lambda sentence: len(word_tokenize(sentence))
longest_sentence_test = max(corpus, key=word_count_test)
length_long_sentence_test = len(word_tokenize(longest_sentence_test))
padded_sentences_test = pad_sequences(embedded_sentences_test, length_long_sentence_test, padding='post')

In [ ]:
#convertimos los tweets DEL TRAIN COMPLETO a numeros
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(list(train.text))
embedded_sentences = word_tokenizer.texts_to_sequences(list(train.text))

#cantidad de palabras unicas
vocab_length = len(word_tokenizer.word_index) + 1

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

In [ ]:
#vinculamos nuestros valores a los de GloVe
import json

embeddings_dictionary = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
  values = line.split()
  word = values[0]
  vector_dimension = np.asarray(values[1:])
  embeddings_dictionary[word] = vector_dimension
f.close()

In [ ]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer_train.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

Modelo 1

In [ ]:
"""model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))"""

"model = Sequential()\nembedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)\nmodel.add(embedding_layer)\nmodel.add(Flatten())\nmodel.add(Dense(1, activation='sigmoid'))"

Modelo 2

In [ ]:
from keras.models import Input, Model
from keras.layers import Bidirectional, GlobalAvgPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, Dense, Embedding, concatenate, Dropout, concatenate

inp1 = Input(shape=(length_long_sentence,))
x = Embedding(embedding_matrix.shape[0], 100, weights=[embedding_matrix])(inp1)
x = Bidirectional(LSTM(256, return_sequences=True))(x)
x = Bidirectional(LSTM(150))(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(64, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)    
model = Model(inputs=inp1, outputs=x)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 31)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 31, 100)           2269800   
_________________________________________________________________
bidirectional (Bidirectional (None, 31, 512)           731136    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               795600    
_________________________________________________________________
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [ ]:
model.fit(padded_sentences_train, y_corpus, epochs=3, verbose=1)

Epoch 1/3
167/167 [==============================] - 79s 473ms/step - loss: 0.4674 - acc: 0.7855
Epoch 2/3
167/167 [==============================] - 79s 472ms/step - loss: 0.3710 - acc: 0.8465
Epoch 3/3
167/167 [==============================] - 81s 483ms/step - loss: 0.2800 - acc: 0.8880


In [ ]:
prediction = model.predict(padded_sentences_test)

In [ ]:
prediction = np.round(prediction)
prediction

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [ ]:
prediction_final = pd.DataFrame({'id':test['id']})
prediction_final['target'] = prediction
prediction_final['target'] = prediction_final['target'].astype(int)
prediction_final.to_csv('prueba_glove_text_completo.csv', index=False)

In [ ]:
prediction_final

,id,target
0,0,0
1,2,1
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,1
3259,10865,1
3260,10868,0
3261,10874,0


#Otra red neuronal

In [ ]:
import numpy as np
import pandas as pd
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

import nltk
import string
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['text_clean'])

train_sequences = tokenizer.texts_to_sequences(train['text_clean'])
train_data = pad_sequences(train_sequences, maxlen=100)

In [ ]:
EMBEDDING_FILE = 'glove.6B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))


In [ ]:
embedding_matrix = np.zeros((50000, 300))
for word, index in tokenizer.word_index.items():
    if index > 50000 - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
model_glove = Sequential()
model_glove.add(Embedding(50000, 300, input_length=100, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
model_glove.fit(train_data, train['target'], epochs = 5)


Epoch 1/5
238/238 [==============================] - 31s 132ms/step - loss: 0.5172 - accuracy: 0.7609
Epoch 2/5
238/238 [==============================] - 31s 132ms/step - loss: 0.4343 - accuracy: 0.8086
Epoch 3/5
238/238 [==============================] - 31s 132ms/step - loss: 0.3728 - accuracy: 0.8426
Epoch 4/5
238/238 [==============================] - 32s 133ms/step - loss: 0.3073 - accuracy: 0.8747
Epoch 5/5
238/238 [==============================] - 31s 132ms/step - loss: 0.2494 - accuracy: 0.9011


In [ ]:
test_sequences = tokenizer.texts_to_sequences(test['text_clean'])
test_data = pad_sequences(test_sequences, maxlen=100)

In [ ]:
predictions= model_glove.predict_classes(test_data)

In [ ]:
predictions

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
pred_glove = pd.DataFrame({'id':test['id']})
pred_glove['target'] = predictions
pred_glove

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
pred_glove.to_csv('prueba_19.csv', index=False)